<a href="https://colab.research.google.com/github/RMNamitha/College-website-design/blob/main/Password_hashing_and_cracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bcrypt argon2-cffi

In [ ]:
# Install required libraries
!pip install bcrypt argon2-cffi

# Import libraries
import hashlib
import bcrypt
from argon2 import PasswordHasher
from itertools import product
import time

# Sample in-memory wordlist for dictionary attacks
WORDLIST_LIST = [
    "123456", "password", "letmein", "qwerty",
    "admin", "secret", "mysecretpassword"
]

# ========================
# HASHING MODULE
# ========================

def hash_sha256(password: str) -> str:
    return hashlib.sha256(password.encode()).hexdigest()

def hash_bcrypt(password: str) -> bytes:
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def verify_bcrypt(password: str, hashed: bytes) -> bool:
    return bcrypt.checkpw(password.encode(), hashed)

def hash_argon2(password: str) -> str:
    ph = PasswordHasher()
    return ph.hash(password)

def verify_argon2(password: str, hashed: str) -> bool:
    ph = PasswordHasher()
    try:
        ph.verify(hashed, password)
        return True
    except:
        return False

# ========================
# ATTACK MODULE
# ========================

def brute_force_sha256(target_hash: str, charset: str, max_len: int) -> str:
    print("\n[+] Starting brute-force (SHA-256)...")
    for length in range(1, max_len + 1):
        for guess in product(charset, repeat=length):
            guess_word = ''.join(guess)
            if hash_sha256(guess_word) == target_hash:
                print(f"[✓] Password found: {guess_word}")
                return guess_word
    print("[x] Password not found.")
    return ""

def dictionary_attack_bcrypt(target_hash: bytes) -> str:
    print("\n[+] Starting dictionary attack (bcrypt)...")
    for word in WORDLIST_LIST:
        if verify_bcrypt(word, target_hash):
            print(f"[✓] Password found: {word}")
            return word
    print("[x] Password not found.")
    return ""

def dictionary_attack_argon2(target_hash: str) -> str:
    print("\n[+] Starting dictionary attack (Argon2)...")
    for word in WORDLIST_LIST:
        if verify_argon2(word, target_hash):
            print(f"[✓] Password found: {word}")
            return word
    print("[x] Password not found.")
    return ""

# ========================
# MAIN FUNCTION
# ========================

def main():
    print("====== PASSWORD HASHING & CRACKING ======\n")
    password = input("Enter a password to hash and test: ")

    print("\n--- Generating hashes ---")
    sha256_hash = hash_sha256(password)
    bcrypt_hash = hash_bcrypt(password)
    argon2_hash = hash_argon2(password)

    print(f"\nSHA-256 Hash: {sha256_hash}")
    print(f"bcrypt Hash : {bcrypt_hash.decode()}")
    print(f"Argon2 Hash : {argon2_hash}")

    # SHA-256 BRUTE FORCE
    charset = 'abc123'
    start = time.time()
    brute_force_sha256(sha256_hash, charset, 4)
    print(f"Time taken: {time.time() - start:.2f}s")

    # bcrypt DICTIONARY ATTACK
    start = time.time()
    dictionary_attack_bcrypt(bcrypt_hash)
    print(f"Time taken: {time.time() - start:.2f}s")

    # Argon2 DICTIONARY ATTACK
    start = time.time()
    dictionary_attack_argon2(argon2_hash)
    print(f"Time taken: {time.time() - start:.2f}s")

# Run the main function
main()


====== PASSWORD HASHING & CRACKING ======

Enter a password to hash and test: Namitha

--- Generating hashes ---

SHA-256 Hash: 5a448b5c17effeb06589e722a218e0624bd93ac8e2d09c76005c4ee804551b3b
bcrypt Hash : $2b$12$p0WymlZ6Izki19EeJnr5oO3MyiT4nkQEp6wKF5BcpUFQHnupyhMfW
Argon2 Hash : $argon2id$v=19$m=65536,t=3,p=4$IPA+dAqTqivUD8/IlTsIcw$shcUZTf9/gOy6KA60PIPODtEHHUv5mp8oijQQWk3zoo

[+] Starting brute-force (SHA-256)...
[x] Password not found.
Time taken: 0.00s

[+] Starting dictionary attack (bcrypt)...
[x] Password not found.
Time taken: 2.30s

[+] Starting dictionary attack (Argon2)...
[x] Password not found.
Time taken: 1.20s
